In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import pandas as pd
import torch
import torch.nn.functional as F
from torch_sparse.tensor import SparseTensor
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score
import logging
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import numpy as np

from src.torch_geo_models import GCN, EdgeWeigher
from src.data.node_classifier.arxiv import load_dataset_pyg, data_to_sparse_symmetric_pyg

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s : %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
torch.cuda.is_available()

True

In [5]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [6]:
dataset = load_dataset_pyg()

In [7]:
data = data_to_sparse_symmetric_pyg(dataset[0])
data.adj_t = data.adj_t.to(device)

In [8]:
split_idx = dataset.get_idx_split()

In [9]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

## Create `base_weights`

In [10]:
edges_gamma_df = pd.read_csv('data/gamma_graph_sage/01-1_gamma_scored_edges.csv')
print(edges_gamma_df.shape)
edges_gamma_df.head()

(2315598, 3)


,source,target,gamma_pred
0,411,0,0.649669
1,640,0,0.784250
2,1162,0,0.787807
3,1897,0,0.787201
4,3396,0,0.784839


In [11]:
base_weights = SparseTensor.from_torch_sparse_coo_tensor(
    torch.sparse_coo_tensor(
        edges_gamma_df[['source', 'target']].values.T,
        edges_gamma_df['gamma_pred'].values,
        size=(data.adj_t.size(0), data.adj_t.size(1))))\
    .to(torch.float32)\
    .to(device)

In [12]:
base_weights.is_symmetric()

True

In [13]:
weigher = EdgeWeigher(base_weights).to(device)
weigher.reset_parameters()

### Model

In [14]:
features = data.x.cuda()
labels = data.y.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [15]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

In [16]:
n_layers = 3
n_iters = 3000
epochs = 100
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001
dropout = 0.5

In [17]:
model = GCN(
    n_layers=n_layers,
    in_channels=input_dim,
    hidden_channels=hidden_channels,
    out_channels=output_dim,
    dropout=dropout,
    batch_norm=True)\
    .to(device)

In [18]:
def train(model, weigher, graph, features, train_mask, optimizer):
    model.train()
    weigher.train()

    optimizer.zero_grad()
    edge_weight = weigher()
    out = model(features, graph, edge_weight=edge_weight)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [19]:
@torch.no_grad()
def test(model, weigher, graph, features, labels, train_mask, val_mask, test_mask, evaluator):
    model.eval()
    weigher.eval()

    edge_weight = weigher()
    out = model(features, graph, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [20]:
evaluator = Evaluator(name='ogbn-arxiv')

In [21]:
optimizer = torch.optim.Adam(
    list(model.parameters()) +
    list(weigher.parameters()),
    lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, weigher, data.adj_t, features, train_mask, optimizer)
    result = test(model, weigher, data.adj_t, features, labels, train_mask, val_mask, test_mask, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.0914, Train: 70.86%, Valid: 70.04% Test: 69.23%


In [24]:
list(weigher.parameters())

[Parameter containing:
 tensor(1., device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor(0., device='cuda:0', requires_grad=True)]